<a href="https://colab.research.google.com/github/iam-pattan/DL-lab/blob/main/Lab_9(Speech).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://github.com/PacktPublishing/Python-Deep-Learning-Cookbook/blob/master/Chapter09/Chapter%209%20-%20Identifying%20speakers%20with%20voice%20recognition.ipynb

In [ ]:
import glob
import numpy as np
import random
import librosa
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
import tensorflow as tf

In [ ]:
# !wget https://zenodo.org/record/1342401/files/Jakobovski/free-spoken-digit-dataset-v1.0.8.zip
# !unzip free-spoken-digit-dataset-v1.0.8.zip

--2021-04-30 10:05:17--  https://zenodo.org/record/1342401/files/Jakobovski/free-spoken-digit-dataset-v1.0.8.zip
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7273063 (6.9M) [application/octet-stream]
Saving to: ‘free-spoken-digit-dataset-v1.0.8.zip.3’

free-spoken-digit-d 100%[===================>]   6.94M  2.83MB/s    in 2.5s    

2021-04-30 10:05:22 (2.83 MB/s) - ‘free-spoken-digit-dataset-v1.0.8.zip.3’ saved [7273063/7273063]

Archive:  free-spoken-digit-dataset-v1.0.8.zip
e9e1155aba3a0c632823b813aa74918807644509
replace Jakobovski-free-spoken-digit-dataset-e9e1155/.gitignore? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
SEED = 123
datapath = "/content/Jakobovski-free-spoken-digit-dataset-e9e1155/recordings/"
'/content/Jakobovski-free-spoken-digit-dataset-e9e1155'

'/content/Jakobovski-free-spoken-digit-dataset-e9e1155'

In [ ]:
files = glob.glob(datapath+"*.wav")
len(files)

1500

In [ ]:
X_train, X_val = train_test_split(files, test_size=0.2, random_state=SEED)

print('# Training examples: {}'.format(len(X_train)))
print('# Validation examples: {}'.format(len(X_val)))

# Training examples: 1200
# Validation examples: 300


In [ ]:
labels = []
for i in range(len(X_train)):
    label = X_train[i].split('/')[-1].split('_')[1]
    if label not in labels:
        labels.append(label)
print(labels)

['nicolas', 'theo', 'jackson']


In [ ]:
label_binarizer = LabelBinarizer()
label_binarizer.fit(list(set(labels)))

def one_hot_encode(x): return label_binarizer.transform(x)

In [ ]:
n_features = 20
max_length = 80
n_classes = len(labels)

In [ ]:
def generator(number):
    i=0
    while i< number:
        i = i+1
        if i%2==0:
            yield i

In [ ]:
datagen = generator(10)

In [ ]:
next(datagen)

2

In [ ]:
def batch_generator(data, batch_size=16):
    while 1:
        random.shuffle(data)
        X, y = [], []
        for i in range(batch_size):
            wav = data[i]
            wave, sr = librosa.load(wav, mono=True)
            label = wav.split('/')[-1].split('_')[1]
            y.append(label)
            mfcc = librosa.feature.mfcc(wave, sr)
            mfcc = np.pad(mfcc, ((0,0), (0, max_length-len(mfcc[0]))), mode='constant', constant_values=0) 
            X.append(np.array(mfcc))
        yield np.array(X), np.array(one_hot_encode(y))

In [ ]:
train_datagen = batch_generator(X_train, batch_size=32)
val_datagen = batch_generator(X_val, batch_size = 4)

In [ ]:
#Model DEFN
input_shape = (n_features, max_length)
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.LSTM(256, return_sequences=True, input_shape=input_shape,
dropout=0.5))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(n_classes, activation='softmax'))

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
callbacks = [tf.keras.callbacks.ModelCheckpoint('SPEAKER_best_model_.hdf5', save_best_only=True),
            tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=2)]

In [ ]:
# help(model.fit)

In [ ]:
history = model.fit(
    train_datagen, 
    steps_per_epoch=10,
    epochs=3,
    validation_data=val_datagen,
    validation_steps=5,
    validation_freq=1,
    verbose=1
    # callbacks = callbacks
)

Epoch 1/3
10/10 [==============================] - 12s 1s/step - loss: 1.0271 - accuracy: 0.4861 - val_loss: 0.4402 - val_accuracy: 0.8000
Epoch 2/3
10/10 [==============================] - 9s 1s/step - loss: 0.5995 - accuracy: 0.7050 - val_loss: 0.4657 - val_accuracy: 0.8000
Epoch 3/3
 4/10 [===========>..................] - ETA: 5s - loss: 0.5102 - accuracy: 0.7715

InvalidArgumentError: ignored

In [ ]:
def testSample(filename,model):
  # Read the WAV file and transform it into mfcc
  print(filename)
  max_length = 80
  wave, sr = librosa.load(filename, mono=True)
  mfcc = librosa.feature.mfcc(wave, sr)
  if(len(mfcc[0]) < 60 ):
    mfcc = np.pad(mfcc, ((0,0), (0, max_length-len(mfcc[0]))), mode='constant', constant_values=0) 
    X = np.array([np.array(mfcc)])
    label = filename.split('/')[-1].split('_')[1]
    pred = model.predict(X)
    pred = np.round(pred,decimals=1)
    pred = label_binarizer.inverse_transform(pred)
    print("Actual Label: ",label)
    print("Predicted Label: ",pred[0])
  else:
    print("Incompatible Audio")

In [ ]:
testSample(files[300],model)

/content/Jakobovski-free-spoken-digit-dataset-e9e1155/recordings/0_nicolas_21.wav
Actual Label:  nicolas
Predicted Label:  nicolas


In [ ]:
# APPENDIX 
# code for loading the entire data into memory as numpy
random.shuffle(files)
X, y = [], []
checkpoint = 0
for i in range(len(files)):
    wav = files[i]
    wave, sr = librosa.load(wav, mono=True)
    mfcc = librosa.feature.mfcc(wave, sr)
    if(len(mfcc[0]) < 80 ):
      mfcc = np.pad(mfcc, ((0,0), (0, max_length-len(mfcc[0]))), mode='constant', constant_values=0) 
      X.append(np.array(mfcc))
      label = wav.split('/')[-1].split('_')[1]
      y.append(label)
      checkpoint+=1
    if(checkpoint%100 ==0):
      print("Current Data count Processed: ",checkpoint)

Current Data count Processed:  100
Current Data count Processed:  200
Current Data count Processed:  300
Current Data count Processed:  400
Current Data count Processed:  500
Current Data count Processed:  600
Current Data count Processed:  700
Current Data count Processed:  800
Current Data count Processed:  900
Current Data count Processed:  1000
Current Data count Processed:  1100
Current Data count Processed:  1200
Current Data count Processed:  1300
Current Data count Processed:  1400


In [ ]:
X = np.array(X)
y = one_hot_encode(y)

# Splitting the dataset into train and validation set 
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)